# SDDP: electric generation examples

## FAST: the hydro-thermal problem

Based on https://odow.github.io/SDDP.jl/latest/examples/FAST_hydro_thermal/

We a consider a toy hydro-thermal example, adapted from https://github.com/leopoldcambier/FAST/blob/master/examples/hydro%20thermal/hydro_thermal.m, where we can produce eletricity either with a thermal plant, either with a hydroelectric plant. The uncertainty is the amount of rainfall at each stage. Rainfall can either be high (10) or small (2), with a uniform probability. At each stage, the demand is equal to 6, and can be satisfied either by burning some fuel, with a price equal to 5 per demand unit, either by using some water, at a negligeable cost. From stage to stage, water can been stored in the reservoir, but there is a tank limit equal to 8. We denote the stored water level at the end of stage 1 by $x_1$ and the used quantity at stage $1$ by $y_1$. The quantity of purchased fuel at stage 1 is denoted by $p_1$, and the initial water quantity in the reservoir is 6.

The problem at stage 1 is then
\begin{align*}
  \min\ & 5 p_1 + V(x_1) \\
  \text{s.t. } & x_1 \leq 8 \\
  & x_1 \leq 6 - y_1 \\
  & p_1 + y_1 \geq 6
\end{align*}
where $V(x_1)$ is the expected second-stage cost, expressed at
\begin{align*}
V(x_1) &= 30 - E\left[5 \min \{ x_1 + \xi, 6 \} \right] \\ 
       &= 30 - 0.5 ( 5 \min \{ x_1+2,6 \} + 5 \min \{ x_1+10,6 \} ) \\
       &= 15 - 5/2 \min \{ x_1+2, 6 \}
\end{align*}

The solution at stage 1 can then trivially be found to be
$$
   x_1 = 0,\ y_1 = 6,\ p_1 = 0
$$
for an expected cost of 10.

In [4]:
using SDDP, HiGHS

In [5]:
using Statistics, Test

In [6]:
model = SDDP.PolicyGraph(
    SDDP.LinearGraph(2),  # Construct the graph with 2 stages
    bellman_function = SDDP.BellmanFunction(lower_bound = 0.0),
    optimizer = HiGHS.Optimizer,
) do sp, t
    # At each stage, the model state is the reservoir level x.
    @variable(sp, 0 <= x <= 8, SDDP.State, initial_value = 0.0)
    @variables(sp, begin
            y >= 0
            p >= 0
            ξ
        end)
    @constraints(sp, begin
            p + y >= 6
            x.out <= x.in - y + ξ
        end)
    
    # We define the rainfall levels.
    # The level is deterministic at stage 1, but can take one of two values at subsequent stages.
    RAINFALL = (t == 1 ? [6] : [2, 10])
    
    SDDP.parameterize(sp, RAINFALL) do ω
        JuMP.fix(ξ, ω)
    end
    @stageobjective(sp, 5 * p)
end

A policy graph with 2 nodes.
 Node indices: 1, 2


In [7]:
det = SDDP.deterministic_equivalent(model, HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 18
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 12 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [9]:
println(det)

Feasibility
Subject to
 x_out - x_in == 0.0
 x_out - x_in == 0.0
 y + p >= 6.0
 y + p >= 6.0
 y + p >= 6.0
 -x_in + x_out + y - ξ <= 0.0
 -x_in + x_out + y - ξ <= 0.0
 -x_in + x_out + y - ξ <= 0.0
 x_in == 0.0
 ξ == 6.0
 ξ == 2.0
 ξ == 10.0
 x_out >= 0.0
 y >= 0.0
 p >= 0.0
 _[6] >= 0.0
 x_out >= 0.0
 y >= 0.0
 p >= 0.0
 _[12] >= 0.0
 x_out >= 0.0
 y >= 0.0
 p >= 0.0
 _[18] >= 0.0
 x_out <= 8.0
 x_out <= 8.0
 _[12] <= 0.0
 x_out <= 8.0
 _[18] <= 0.0



In [6]:
JuMP.optimize!(det)
@test JuMP.objective_value(det) == 10

Presolving model
6 rows, 7 cols, 12 nonzeros
6 rows, 7 cols, 12 nonzeros
Presolve : Reductions: rows 6(-2); columns 7(-11); elements 12(-10)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(18) 0s
          5     1.0000000000e+01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  1.0000000000e+01
HiGHS run time      :          0.01


Test Passed

In [10]:
SDDP.train(model, iteration_limit = 10, log_frequency = 5)
@test SDDP.calculate_bound(model) == 10

------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-21

Problem
  Nodes           : 2
  State variables : 1
  Scenarios       : 2.00000e+00
  Existing cuts   : false
  Subproblem structure                      : (min, max)
    Variables                               : (6, 6)
    VariableRef in MOI.EqualTo{Float64}     : (1, 1)
    VariableRef in MOI.LessThan{Float64}    : (1, 2)
    AffExpr in MOI.LessThan{Float64}        : (1, 1)
    AffExpr in MOI.GreaterThan{Float64}     : (1, 1)
    VariableRef in MOI.GreaterThan{Float64} : (4, 4)
Options
  Solver          : serial mode
  Risk measure    : SDDP.Expectation()
  Sampling scheme : SDDP.InSampleMonteCarlo

Numerical stability report
  Non-zero Matrix range     [1e+00, 1e+00]
  Non-zero Objective range  [1e+00, 5e+00]
  Non-zero Bounds range     [8e+00, 8e+00]
  Non-zero RHS range        [6e+00, 6e+00]
No problems detected

 Iteration    Simulation       Bou

Test Passed

## Hydro-thermal scheduling

Copy-paste from https://odow.github.io/SDDP.jl/latest/examples/Hydro_thermal/#Hydro-thermal-scheduling

### Problem Description

In a hydro-thermal problem, the agent controls a hydro-electric generator and reservoir. Each time period, they need to choose a generation quantity from thermal $g_t$, and hydro $g_h$, in order to meet demand $w_d$, which is a stagewise-independent random variable. The state variable, $x$, is the quantity of water in the reservoir at the start of each time period, and it has a minimum level of 5 units and a maximum level of 15 units. We assume that there are 10 units of water in the reservoir at the start of time, so that $x_0$ = 10. The state-variable is connected through time by the water balance constraint:
$$x.out = x.in - g_h - s + w_i,$$
where $x.out$ is the quantity of water at the end of the time period, $x.in$ is the quantity of water at the start of the time period, $s$ is the quantity of water spilled from the reservoir, and $w_i$ is a stagewise-independent random variable that represents the inflow into the reservoir during the time period.

We assume that there are three stages, $t$=1, 2, 3, representing summer-fall, winter, and spring, and that we are solving this problem in an infinite-horizon setting with a discount factor of 0.95.

In each stage, the agent incurs the cost of spillage, plus the cost of thermal generation. We assume that the cost of thermal generation is dependent on the stage $t$ = 1, 2, 3, and that in each stage, $w$ is drawn from the set $(w_i, w_d)$ = $\{(0, 7.5), (3, 5), (10, 2.5)\}$ with equal probability.

### Constructing the policy graph

There are three stages in our problem, so we construct a linear policy graph with three stages using `SDDP.LinearGraph`:

In [11]:
graph = SDDP.LinearGraph(3)

Root
 0
Nodes
 1
 2
 3
Arcs
 0 => 1 w.p. 1.0
 1 => 2 w.p. 1.0
 2 => 3 w.p. 1.0


Then, because we want to solve an infinite-horizon problem, we add an additional edge between node 3 and node 1 with probability 0.95:

In [12]:
SDDP.add_edge(graph, 3 => 1, 0.95)

### Constructing the model

The state variable $x$, constructed by passing the `SDDP.State` tag to @variable is actually a Julia struct with two fields: $x.in$ and $x.out$ corresponding to the incoming and outgoing state variables respectively. Both $x.in$ and $x.out$ are standard JuMP variables. The initial_value keyword provides the value of the state variable in the root node (i.e., $x_0$).

Compared to a JuMP model, one key difference is that we use @stageobjective instead of @objective. The `SDDP.parameterize` function takes a list of supports for $w$ and parameterizes the JuMP model `sp` by setting the right-hand sides of the appropriate constraints (note how the constraints initially have a right-hand side of 0). By default, it is assumed that the realizations have uniform probability, but a probability mass vector can also be provided.

In [13]:
model = SDDP.PolicyGraph(
    graph,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
) do sp, t
    @variable(sp, 5 <= x <= 15, SDDP.State, initial_value = 10)
    @variable(sp, g_t >= 0)
    @variable(sp, g_h >= 0)
    @variable(sp, s >= 0)
    @constraint(sp, balance, x.out - x.in + g_h + s == 0)
    @constraint(sp, demand, g_h + g_t == 0)
    @stageobjective(sp, s + t * g_t)
    SDDP.parameterize(sp, [[0, 7.5], [3, 5], [10, 2.5]]) do w
        set_normalized_rhs(balance, w[1])
        return set_normalized_rhs(demand, w[2])
    end
end

A policy graph with 3 nodes.
 Node indices: 1, 2, 3


### Training the policy

Once a model has been constructed, the next step is to train the policy. This can be achieved using `SDDP.train`. There are many options that can be passed, but `iteration_limit` terminates the training after the prescribed number of SDDP iterations.

In [15]:
SDDP.train(model, iteration_limit = 100)

------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-21

Problem
  Nodes           : 3
  State variables : 1
  Scenarios       : Inf
  Existing cuts   : true
  Subproblem structure                      : (min, max)
    Variables                               : (6, 6)
    VariableRef in MOI.EqualTo{Float64}     : (1, 1)
    VariableRef in MOI.LessThan{Float64}    : (1, 1)
    AffExpr in MOI.GreaterThan{Float64}     : (9, 13)
    AffExpr in MOI.EqualTo{Float64}         : (2, 2)
    VariableRef in MOI.GreaterThan{Float64} : (5, 5)
Options
  Solver          : serial mode
  Risk measure    : SDDP.Expectation()
  Sampling scheme : SDDP.InSampleMonteCarlo

Numerical stability report
  Non-zero Matrix range     [2e-03, 2e+00]
  Non-zero Objective range  [1e+00, 3e+00]
  Non-zero Bounds range     [5e+00, 2e+01]
  Non-zero RHS range        [2e+00, 2e+02]
No problems detected

 Iteration    Simulation       Bound      

┌ Warning: Re-training a model with existing cuts!
│ 
│ Are you sure you want to do this? The output from this training may be
│ misleading because the policy is already partially trained.
│ 
│ If you meant to train a new policy with different settings, you must
│ build a new model.
│ 
│ If you meant to refine a previously trained policy, turn off this
│ warning by passing `add_to_existing_cuts = true` as a keyword argument
│ to `SDDP.train`.
│ 
│ In a future release, this warning may turn into an error.
└ @ SDDP C:\Users\slash\.julia\packages\SDDP\VpZOu\src\algorithm.jl:931


        1    4.105195e+02   2.364194e+02   2.760000e-01          1      24315
        2    2.500329e+02   2.364199e+02   4.250000e-01          1      24546
        3    8.999757e+02   2.364245e+02   1.009000e+00          1      25485
        4    4.092475e+02   2.364252e+02   1.279000e+00          1      25884
        5    2.079988e+02   2.364260e+02   1.415000e+00          1      26091
        6    2.229906e+02   2.364265e+02   1.551000e+00          1      26322
        7    9.750000e+01   2.364273e+02   1.658000e+00          1      26469
        8    7.810577e+02   2.364279e+02   2.198000e+00          1      27312
        9    4.674973e+02   2.364292e+02   2.502000e+00          1      27819
       10    1.300208e+02   2.364297e+02   2.633000e+00          1      28002
       11    5.084974e+02   2.364303e+02   2.884000e+00          1      28377
       12    5.459912e+02   2.364306e+02   3.244000e+00          1      28932
       13    1.520002e+02   2.364307e+02   3.391000e+00         

### Simulating the policy

After training, we can simulate the policy using `SDDP.simulate`.

In [16]:
sims = SDDP.simulate(model, 100, [:g_t])
mu = round(mean([s[1][:g_t] for s in sims]), digits = 2)
println("On average, $(mu) units of thermal are used in the first stage.")

On average, 2.02 units of thermal are used in the first stage.


### Extracting the water values

Finally, we can use `SDDP.ValueFunction` and `SDDP.evaluate` to obtain and evaluate the value function at different points in the state-space. Note that since we are minimizing, the price has a negative sign: each additional unit of water leads to a decrease in the the expected long-run cost.

In [17]:
V = SDDP.ValueFunction(model[1])
cost, price = SDDP.evaluate(V, x = 10)

Presolving model
382 rows, 1 cols, 382 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4015); columns 0(-2); elements 0(-8030) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  2.3356257439e+02
HiGHS run time      :          0.02


(233.562574389762, Dict(:x => -0.6600005524119096))